## Merck FEP+ dataset formatting

##### Inputs
- ```.pdb``` file containing protein structures

- ```.sdf``` file containing ligand names, coordinates and DeltaG values


##### Output

- ```.pdb``` file for each ligand

- Train, validation and test data files for aescore with the following format:

        DeltaG path/to/protein.pdb path/to/ligand.pdb




In [162]:
import rdkit
from rdkit import Chem
import os
import pickle
import pandas as pd
import numpy as np

In [163]:
def getDirNames(path):
    return [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

In [164]:
def getLigandDataFromSDF(sdf_file, aff_props, id_prop='_Name', removeHs=False):
    """
    Read in a SDF file and return a nested dictionary of ligand data.

    Parameters
    ----------
    sdf_file : str 
        Path to SDF file.
    properties : list
        List of properties to extract from SDF file.
    removeHs : bool, optional
        Remove hydrogens from the ligand molecules. Default is False.
    """

    suppl = Chem.SDMolSupplier(sdf_file,removeHs=removeHs)
    lig_objs = [m for m in suppl if m is not None]
    lig_data = {}

    for m in lig_objs:

        id = m.GetProp(id_prop)
        id = formatTargetName(id)

        for aff_prop in aff_props:
            try:
                aff = float(m.GetProp(aff_prop))
                break
            except:
                continue

        lig_data[id] = {aff_prop: aff, 'rdmol': m}
        
    return lig_data

In [165]:
def molToPDB(lig_data, target_name, outdir):
    """
    Write out a PDB file for each molecule in a dictionary of molecule data. 
    """
    saved = total = 0

    for lig_id in lig_data.keys():
        mol = lig_data[lig_id]['rdmol']
        try:
            Chem.MolToPDBFile(mol, '{}/{}_ligand_{}.pdb'.format(outdir, target_name, lig_id))
            saved += 1
        except:
            print('Error writing {} to PDB file'.format(lig_id))
        
        total += 1
    
    print('{} out of {} {} ligands saved to {}'.format(saved, total, target_name, outdir))

### Creating a dictionary of ligand data and ligand PDB files

In [179]:
targets = getDirNames('../data')
mol_data_dict = {target: getLigandDataFromSDF(sdf_file=f'../data/{target}/{target}_ligands.sdf',
                                              aff_props=['IC50 uM', 'IC50[uM]','IC50[uM](SPA)','IC50[nM]']) for target in targets}

In [180]:
for target in mol_data_dict:
    molToPDB(mol_data_dict[target], target, '../data/{}'.format(target))

pickle.dump(mol_data_dict, open('../data/all_ligands_mol_dict.pkl', 'wb'))

40 out of 40 pfkfb3 ligands saved to ../data/pfkfb3
33 out of 33 cdk8 ligands saved to ../data/cdk8
26 out of 26 shp2 ligands saved to ../data/shp2
28 out of 28 eg5_alternativeloop ligands saved to ../data/eg5_alternativeloop
42 out of 42 hif2a ligands saved to ../data/hif2a
44 out of 44 syk ligands saved to ../data/syk
27 out of 27 tnks2 ligands saved to ../data/tnks2
24 out of 24 cmet ligands saved to ../data/cmet
28 out of 28 eg5 ligands saved to ../data/eg5


### Read in dG values

In [181]:
def formatTargetName(name):
    
    if isinstance(name, str):
        
        if ' ' in name:
            name = name.replace(' ', '_')
        
        if '/' in name:
            name = name.replace('/', '_')
        
        if '.0' in name:
            name = name.replace('.0', '')
    
    if isinstance(name, float):
        return str(int(name))

    return name

In [182]:
def findFile(path, search_string):
    """
    Find a file in a directory that contains a given string.
    """
    matches = []
    for file in os.listdir(path):
        if search_string in file:
            matches.append(file)

    if len(matches) == 0:
        raise ValueError(f'No files found containing string "{search_string}"')
    
    elif len(matches) > 1:
        raise ValueError(f'Multiple files found containing string "{search_string}"')
    
    else:
        return matches[0]

In [183]:
targets = getDirNames('../data')
mol_data_dict = pickle.load(open('../data/all_ligands_mol_dict.pkl', 'rb'))

for target in targets:
    if target == 'eg5_alternativeloop':
        continue
    else:
        fep_data = pd.read_csv(f'../data/{target}/results_20ns.csv')
        
        for i, row in fep_data.iterrows():
            ligand = formatTargetName(row['Ligand'])

            try:
                mol_data_dict[target][ligand]['Exp. dG'] = row['Exp. ΔG']
                mol_data_dict[target][ligand]['Pred. dG'] = row['Pred. ΔG']
                mol_data_dict[target][ligand]['Pred. Error'] = row['Pred. Error']

            except:
                print(f'Error processing {target} {ligand}')

pickle.dump(mol_data_dict, open('../data/all_ligands_mol_dict.pkl', 'wb'))

In [184]:
mol_data_dict

{'pfkfb3': {'43': {'IC50[uM]': 1.373,
   'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f9c0f899d50>,
   'Exp. dG': -8.0,
   'Pred. dG': -8.82,
   'Pred. Error': 0.11},
  '30': {'IC50[uM]': 0.181,
   'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f9c11163ab0>,
   'Exp. dG': -9.2,
   'Pred. dG': -9.41,
   'Pred. Error': 0.1},
  '65': {'IC50[uM]': 0.111,
   'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f9c1091aac0>,
   'Exp. dG': -9.49,
   'Pred. dG': -9.34,
   'Pred. Error': 0.09},
  '49': {'IC50[uM]': 1.528,
   'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f9c0f89a110>,
   'Exp. dG': -7.93,
   'Pred. dG': -6.68,
   'Pred. Error': 0.1},
  '24': {'IC50[uM]': 0.315,
   'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f9c0f89bf60>,
   'Exp. dG': -8.87,
   'Pred. dG': -9.79,
   'Pred. Error': 0.11},
  '67': {'IC50[uM]': 0.031,
   'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f9c0f899c10>,
   'Exp. dG': -10.24,
   'Pred. dG': -10.57,
   'Pred. Error': 0.09},
  '62': {'IC50[uM]': 2.301,
   'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f9c0f89a020>,
   

In [192]:
mol_dict = pickle.load(open('../data/all_ligands_mol_dict.pkl', 'rb'))

loc_dict = dict()

for target in mol_dict.keys():
    
    if target == 'eg5_alternativeloop':
        continue
    
    target_pdb = findFile(f"../data/{target}", "prepared" )
    target_loc = '../data/{}/{}'.format(target, target_pdb)
    
    for ligand in mol_dict[target]:
        exp_dg = mol_dict[target][ligand]['Exp. dG']
        pred_dg = mol_dict[target][ligand]['Pred. dG']
        pred_error = mol_dict[target][ligand]['Pred. Error']

        ligand_loc = '../data/{}/{}_ligand_{}.pdb'.format(target, target, ligand)
        entry = {'target':target, 'target_loc':target_loc, 'ligand':ligand, 'ligand_loc':ligand_loc, 'Exp. dG':exp_dg, 'Pred. dG':pred_dg, 'Pred. Error':pred_error}
        loc_dict['{}_{}'.format(target,ligand)] = entry

loc_df = pd.DataFrame.from_dict(data=loc_dict, orient='index')
pickle.dump(loc_df, open('../data/all_ligands_loc_df.pkl', 'wb'))
loc_df.to_csv('../data/all_ligands_data.csv')


In [195]:
loc_df.target.unique()

array(['pfkfb3', 'cdk8', 'shp2', 'hif2a', 'syk', 'tnks2', 'cmet', 'eg5'],
      dtype=object)

### Creating input files for AEScore

In [10]:
def stratifiedSample(df, col, n_samples):
    """
    Draw a sample from a dataframe, such that the sample is stratified based on the frequency of values in a column.
    """
    return df.groupby(col, group_keys=False).apply(lambda x: x.sample(int(np.rint(n_samples*len(x)/len(df))))).sample(frac=1)


In [11]:
loc_df = pickle.load(open('../data/all_ligands_loc_df.pkl', 'rb'))
loc_df = loc_df.round({'dG': 2})

val_set = stratifiedSample(loc_df, 'target', 20)
test_set = stratifiedSample(loc_df[~loc_df.index.isin(val_set.index)], 'target', 20)
train_set = loc_df[~loc_df.index.isin(val_set.index) & ~loc_df.index.isin(test_set.index)]

In [12]:
overlap = [i for i in train_set.index if i in val_set.index or i in test_set.index]
print(bool(overlap), len(overlap))

False 0


In [13]:
val_set[['dG','target_loc','ligand_loc']].to_csv('../data/valid.dat', header=False, index=False, sep=' ')
test_set[['dG','target_loc','ligand_loc']].to_csv('../data/test.dat', header=False, index=False, sep=' ')
train_set[['dG','target_loc','ligand_loc']].to_csv('../data/train.dat', header=False, index=False, sep=' ')